In [1]:
import csv
import sys
import re
import matplotlib.pyplot as plt
import numpy as np

# Cutoff for rare variants 

In [2]:
cutoff  = 0.000001

In [3]:
def writelist (mylist, myfile): 
    fp =  open(myfile,'a+')
    for idx,item in enumerate(mylist):
            fp.write(item)
            fp.write('\t')
    fp.write('\n')
    fp.close()

In [4]:
def intersections(list1, list2):
    intersection_list = []
    for i in list1: 
        if i in list2: 
                intersection_list.append(i)
    return intersection_list

In [5]:
def difference(list1, list2):
    unique_list = []
    for i in list1: 
        if i not in list2: 
                unique_list.append(i)
    return unique_list

In [6]:
def exist (list1, list2): 
    found_list = []
    for i in list1: 
            if i not in found_list: 
                    found_list.append(i)
    for i in list2: 
            if i not in found_list: 
                    found_list.append(i)
    return found_list 

In [7]:
def common_variants(variants1, variants2, outfile): 
        fp = open(outfile, 'w')
        for i in variants1: 
               for j in variants2: 
                     count = 0 
                     for a in range(0,4): 
                            if i[a] == j[a]:
                                count +=1
                     if count == 4: 
                          for item in (i):
                                fp.write(item)
                                fp.write('\t')
                          fp.write('\n')

# Read annovar annotation file

In [8]:
 def load_annovar(filename):
    rows = []
    exacall = []
    gnomall = []
    rs = []
    gene = []
    pathogenics = []
    functions = []
    exonicfunctions =[]
    fp = open(filename, 'r')
    r = csv.reader(fp, delimiter='\t')
    for n, row in enumerate(r):
        if n == 0: 
                for i in range(0,len(row) ): 
                     print (i,row[i])
                continue
        exonicfunctions.append(row[5])
        exacall.append(row[11])
        gnomall.append(row[119])
        rs.append(row[19])
        rows.append(row)
        gene.append(row[6])
        functions.append(row[8])
        pathogenics.append(row[54])
        if row[6] == 'GBA':
                print('genes, Exac and Gnom are:', row[6], row[11], row[119])
    return rows, functions, exonicfunctions, rs, gene,  exacall , gnomall, pathogenics

# Filter for rare MAF

In [9]:
def maf_filter(mfile): 
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    outfile = mfile.split('.')[0] + '_raremaf.txt'
    raremaf_genes = []
    variants_list = [] 
    for i in range(0,len(exacall) ): 
        if exfunc[i] =='exonic':
                    #========================================
                    #If both gnomAD and ExAC is '.'
                    #========================================
                    if (gnomall[i] != '.') and (exacall[i] !='.'):
                        if   (float(gnomall[i]) < cutoff) and (float(exacall[i]) < cutoff) : 
                            count += 1 
                            writelist(rows[i], outfile)
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in raremaf_genes: 
                                    raremaf_genes.append(gene[i])
                    #========================================
                    #If both gnomAD is '.' and ExAC is not '.'
                    #========================================               
                    elif (gnomall[i] == '.') and (exacall[i] != '.') and (float(exacall[i]) < cutoff) :
                            count += 1 
                            writelist(rows[i], outfile)                    
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in raremaf_genes: 
                                    raremaf_genes.append(gene[i])
                    #========================================
                    #If both ExAC is '.' and gnomAD is not '.'
                    #======================================== 
                    elif (exacall[i] =='.') and (gnomall[i] != '.') and (float (gnomall[i]) < cutoff)  :
                            count += 1 
                            writelist(rows[i], outfile)
                            variants_list.append(rows[i])
                            print(rs[i], exfunc[i],gene[i], exacall[i],gnomall[i])
                            if gene[i] not in  raremaf_genes: 
                                     raremaf_genes.append(gene[i])
    return(raremaf_genes,  variants_list)

# Filter for pathogenic variants

In [10]:
def pathogenic_filter(mfile):
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    pathogenic_genes = []    
    variants_list = [] 
    outfile = mfile.split('.')[0]+'pathogenic.txt'
    for i in range(0,len(exacall) ): 
        if 'athogenic' in pathogenics[i]: 
            if (exfunc[i] =='exonic'): 
                if ((exacall[i] == '.') or( float(exacall[i]) < cutoff)) and  ((gnomall[i] =='.') or (float(gnomall[i]) < cutoff)):
                        count += 1 
                        writelist(rows[i], outfile)
                        variants_list.append(rows[i])
                        print(rs[i], gene[i], exfunc[i],exacall[i],gnomall[i], pathogenics[i])
                        if gene[i] not in pathogenic_genes:
                                pathogenic_genes.append(gene[i])
    print ('Number of variants that are rare and pathogenic cutoff <', cutoff, 'is:',count)
    return pathogenic_genes, variants_list 

# Filter for no MAF variants

In [11]:
def nomaf_filter(mfile):    
    rows, functions,exfunc, rs,gene, exacall, gnomall, pathogenics = load_annovar(mfile)
    count = 0
    nomafs_genes = []
    nomafvariants = [] 
    outfile = mfile.split('.')[0]+'nomaf.txt'
    
    for i in range(0,len(exacall) ): 
        if (exacall[i] == '.') and (gnomall[i] =='.') and exfunc[i] =='exonic':
                count += 1 
                writelist(rows[i], outfile)
                if gene[i] not in nomafs_genes:
                        nomafs_genes.append(gene[i])
                var = rs[i]+'.'+exfunc[i]+'.'+gene[i]
                if var not in  nomafvariants: 
                         nomafvariants.append(var)
    print ('Number of variants with no MAf in both exacall and gnomAD <', cutoff, 'is:',count)
    return nomafs_genes,  nomafvariants 

# F366 Analysis

In [12]:
f366m1_raremaf, f366m1_variants  = maf_filter('f366m1-anno-ref.hg19_multianno.txt')

(0, 'Chr')
(1, 'Start')
(2, 'End')
(3, 'Ref')
(4, 'Alt')
(5, 'Func.refGene')
(6, 'Gene.refGene')
(7, 'GeneDetail.refGene')
(8, 'ExonicFunc.refGene')
(9, 'AAChange.refGene')
(10, 'cytoBand')
(11, 'ExAC_ALL')
(12, 'ExAC_AFR')
(13, 'ExAC_AMR')
(14, 'ExAC_EAS')
(15, 'ExAC_FIN')
(16, 'ExAC_NFE')
(17, 'ExAC_OTH')
(18, 'ExAC_SAS')
(19, 'avsnp147')
(20, 'SIFT_score')
(21, 'SIFT_pred')
(22, 'Polyphen2_HDIV_score')
(23, 'Polyphen2_HDIV_pred')
(24, 'Polyphen2_HVAR_score')
(25, 'Polyphen2_HVAR_pred')
(26, 'LRT_score')
(27, 'LRT_pred')
(28, 'MutationTaster_score')
(29, 'MutationTaster_pred')
(30, 'MutationAssessor_score')
(31, 'MutationAssessor_pred')
(32, 'FATHMM_score')
(33, 'FATHMM_pred')
(34, 'PROVEAN_score')
(35, 'PROVEAN_pred')
(36, 'VEST3_score')
(37, 'CADD_raw')
(38, 'CADD_phred')
(39, 'DANN_score')
(40, 'fathmm-MKL_coding_score')
(41, 'fathmm-MKL_coding_pred')
(42, 'MetaSVM_score')
(43, 'MetaSVM_pred')
(44, 'MetaLR_score')
(45, 'MetaLR_pred')
(46, 'integrated_fitCons_score')
(47, 'integrat

In [14]:
f366m2_raremaf , f366m2_variants= maf_filter('f366m2-anno-ref.hg19_multianno.txt')

(0, 'Chr')
(1, 'Start')
(2, 'End')
(3, 'Ref')
(4, 'Alt')
(5, 'Func.refGene')
(6, 'Gene.refGene')
(7, 'GeneDetail.refGene')
(8, 'ExonicFunc.refGene')
(9, 'AAChange.refGene')
(10, 'cytoBand')
(11, 'ExAC_ALL')
(12, 'ExAC_AFR')
(13, 'ExAC_AMR')
(14, 'ExAC_EAS')
(15, 'ExAC_FIN')
(16, 'ExAC_NFE')
(17, 'ExAC_OTH')
(18, 'ExAC_SAS')
(19, 'avsnp147')
(20, 'SIFT_score')
(21, 'SIFT_pred')
(22, 'Polyphen2_HDIV_score')
(23, 'Polyphen2_HDIV_pred')
(24, 'Polyphen2_HVAR_score')
(25, 'Polyphen2_HVAR_pred')
(26, 'LRT_score')
(27, 'LRT_pred')
(28, 'MutationTaster_score')
(29, 'MutationTaster_pred')
(30, 'MutationAssessor_score')
(31, 'MutationAssessor_pred')
(32, 'FATHMM_score')
(33, 'FATHMM_pred')
(34, 'PROVEAN_score')
(35, 'PROVEAN_pred')
(36, 'VEST3_score')
(37, 'CADD_raw')
(38, 'CADD_phred')
(39, 'DANN_score')
(40, 'fathmm-MKL_coding_score')
(41, 'fathmm-MKL_coding_pred')
(42, 'MetaSVM_score')
(43, 'MetaSVM_pred')
(44, 'MetaLR_score')
(45, 'MetaLR_pred')
(46, 'integrated_fitCons_score')
(47, 'integrat

In [180]:
f366m1_pathogenic,f366m1_pathogenic_variants   = pathogenic_filter('f366m1-anno-ref.hg19_multianno.txt')

(0, 'Chr')
(1, 'Start')
(2, 'End')
(3, 'Ref')
(4, 'Alt')
(5, 'Func.refGene')
(6, 'Gene.refGene')
(7, 'GeneDetail.refGene')
(8, 'ExonicFunc.refGene')
(9, 'AAChange.refGene')
(10, 'cytoBand')
(11, 'ExAC_ALL')
(12, 'ExAC_AFR')
(13, 'ExAC_AMR')
(14, 'ExAC_EAS')
(15, 'ExAC_FIN')
(16, 'ExAC_NFE')
(17, 'ExAC_OTH')
(18, 'ExAC_SAS')
(19, 'avsnp147')
(20, 'SIFT_score')
(21, 'SIFT_pred')
(22, 'Polyphen2_HDIV_score')
(23, 'Polyphen2_HDIV_pred')
(24, 'Polyphen2_HVAR_score')
(25, 'Polyphen2_HVAR_pred')
(26, 'LRT_score')
(27, 'LRT_pred')
(28, 'MutationTaster_score')
(29, 'MutationTaster_pred')
(30, 'MutationAssessor_score')
(31, 'MutationAssessor_pred')
(32, 'FATHMM_score')
(33, 'FATHMM_pred')
(34, 'PROVEAN_score')
(35, 'PROVEAN_pred')
(36, 'VEST3_score')
(37, 'CADD_raw')
(38, 'CADD_phred')
(39, 'DANN_score')
(40, 'fathmm-MKL_coding_score')
(41, 'fathmm-MKL_coding_pred')
(42, 'MetaSVM_score')
(43, 'MetaSVM_pred')
(44, 'MetaLR_score')
(45, 'MetaLR_pred')
(46, 'integrated_fitCons_score')
(47, 'integrat

In [15]:
f366m2_pathogenic,f366m2_pathogenic_variants = pathogenic_filter('f366m2-anno-ref.hg19_multianno.txt')

(0, 'Chr')
(1, 'Start')
(2, 'End')
(3, 'Ref')
(4, 'Alt')
(5, 'Func.refGene')
(6, 'Gene.refGene')
(7, 'GeneDetail.refGene')
(8, 'ExonicFunc.refGene')
(9, 'AAChange.refGene')
(10, 'cytoBand')
(11, 'ExAC_ALL')
(12, 'ExAC_AFR')
(13, 'ExAC_AMR')
(14, 'ExAC_EAS')
(15, 'ExAC_FIN')
(16, 'ExAC_NFE')
(17, 'ExAC_OTH')
(18, 'ExAC_SAS')
(19, 'avsnp147')
(20, 'SIFT_score')
(21, 'SIFT_pred')
(22, 'Polyphen2_HDIV_score')
(23, 'Polyphen2_HDIV_pred')
(24, 'Polyphen2_HVAR_score')
(25, 'Polyphen2_HVAR_pred')
(26, 'LRT_score')
(27, 'LRT_pred')
(28, 'MutationTaster_score')
(29, 'MutationTaster_pred')
(30, 'MutationAssessor_score')
(31, 'MutationAssessor_pred')
(32, 'FATHMM_score')
(33, 'FATHMM_pred')
(34, 'PROVEAN_score')
(35, 'PROVEAN_pred')
(36, 'VEST3_score')
(37, 'CADD_raw')
(38, 'CADD_phred')
(39, 'DANN_score')
(40, 'fathmm-MKL_coding_score')
(41, 'fathmm-MKL_coding_pred')
(42, 'MetaSVM_score')
(43, 'MetaSVM_pred')
(44, 'MetaLR_score')
(45, 'MetaLR_pred')
(46, 'integrated_fitCons_score')
(47, 'integrat

In [16]:
f366m1_nomaf = nomaf_filter('f366m1-anno-ref.hg19_multianno.txt')

(0, 'Chr')
(1, 'Start')
(2, 'End')
(3, 'Ref')
(4, 'Alt')
(5, 'Func.refGene')
(6, 'Gene.refGene')
(7, 'GeneDetail.refGene')
(8, 'ExonicFunc.refGene')
(9, 'AAChange.refGene')
(10, 'cytoBand')
(11, 'ExAC_ALL')
(12, 'ExAC_AFR')
(13, 'ExAC_AMR')
(14, 'ExAC_EAS')
(15, 'ExAC_FIN')
(16, 'ExAC_NFE')
(17, 'ExAC_OTH')
(18, 'ExAC_SAS')
(19, 'avsnp147')
(20, 'SIFT_score')
(21, 'SIFT_pred')
(22, 'Polyphen2_HDIV_score')
(23, 'Polyphen2_HDIV_pred')
(24, 'Polyphen2_HVAR_score')
(25, 'Polyphen2_HVAR_pred')
(26, 'LRT_score')
(27, 'LRT_pred')
(28, 'MutationTaster_score')
(29, 'MutationTaster_pred')
(30, 'MutationAssessor_score')
(31, 'MutationAssessor_pred')
(32, 'FATHMM_score')
(33, 'FATHMM_pred')
(34, 'PROVEAN_score')
(35, 'PROVEAN_pred')
(36, 'VEST3_score')
(37, 'CADD_raw')
(38, 'CADD_phred')
(39, 'DANN_score')
(40, 'fathmm-MKL_coding_score')
(41, 'fathmm-MKL_coding_pred')
(42, 'MetaSVM_score')
(43, 'MetaSVM_pred')
(44, 'MetaLR_score')
(45, 'MetaLR_pred')
(46, 'integrated_fitCons_score')
(47, 'integrat

In [183]:
f366m2_nomaf = nomaf_filter('f366m2-anno-ref.hg19_multianno.txt')

(0, 'Chr')
(1, 'Start')
(2, 'End')
(3, 'Ref')
(4, 'Alt')
(5, 'Func.refGene')
(6, 'Gene.refGene')
(7, 'GeneDetail.refGene')
(8, 'ExonicFunc.refGene')
(9, 'AAChange.refGene')
(10, 'cytoBand')
(11, 'ExAC_ALL')
(12, 'ExAC_AFR')
(13, 'ExAC_AMR')
(14, 'ExAC_EAS')
(15, 'ExAC_FIN')
(16, 'ExAC_NFE')
(17, 'ExAC_OTH')
(18, 'ExAC_SAS')
(19, 'avsnp147')
(20, 'SIFT_score')
(21, 'SIFT_pred')
(22, 'Polyphen2_HDIV_score')
(23, 'Polyphen2_HDIV_pred')
(24, 'Polyphen2_HVAR_score')
(25, 'Polyphen2_HVAR_pred')
(26, 'LRT_score')
(27, 'LRT_pred')
(28, 'MutationTaster_score')
(29, 'MutationTaster_pred')
(30, 'MutationAssessor_score')
(31, 'MutationAssessor_pred')
(32, 'FATHMM_score')
(33, 'FATHMM_pred')
(34, 'PROVEAN_score')
(35, 'PROVEAN_pred')
(36, 'VEST3_score')
(37, 'CADD_raw')
(38, 'CADD_phred')
(39, 'DANN_score')
(40, 'fathmm-MKL_coding_score')
(41, 'fathmm-MKL_coding_pred')
(42, 'MetaSVM_score')
(43, 'MetaSVM_pred')
(44, 'MetaLR_score')
(45, 'MetaLR_pred')
(46, 'integrated_fitCons_score')
(47, 'integrat

In [184]:
common_variants(f366m1_variants,f366m2_variants, 'f366m1m2.rare.common.txt')
common_variants(f366m1_pathogenic_variants,f366m2_pathogenic_variants, 'f366m1m2.pathogenic.common.txt')

In [185]:
f366m1_m2_raremaf = intersections(f366m2_raremaf,f366m1_raremaf)
print(f366m1_m2_raremaf)

['IGSF3', 'IGFN1', 'TEKT4', 'ANKRD36C', 'ADRA2B', 'SYN2', 'PLCL2', 'FRG2C', 'ZNF717', 'ADRA2C', 'KIAA1211', 'FOXO3', 'MUC3A', 'MUC17', 'KBTBD11', 'PCMTD1', 'MEGF9', 'CTBP2', 'MUC6', 'MUC5B', 'MTCH2', 'MAML2', 'VWF', 'CLECL1', 'GXYLT1', 'FAM186A', 'KRT1', 'TDG', 'MMP17', 'PABPC3', 'ERICH6B', 'GOLGA6L2', 'BNC1', 'APOBR', 'SULT1A2', 'HYDIN', 'KRTAP9-1', 'CDC27', 'CABLES1', 'MUC16', 'ZNF681', 'NUMBL', 'ZNF534', 'KIR3DL3', 'KIR2DL4', 'KIR3DL1', 'KIR2DS4', 'PLAC4', 'FAM47A', 'SLC25A5']


# F366 pathogenic variants common between M1 and M2

In [186]:
f366m1_m2_pathogenic = intersections(f366m1_pathogenic , f366m2_pathogenic )
print(f366m1_m2_pathogenic )

['GBA']


# F366  no MAFs common between M1 and M2

In [187]:
f366m1_m2_nomaf   = difference(f366m1_nomaf  , f366m2_nomaf  )
print(f366m1_m2_nomaf )

[['ACAP3', 'PUSL1', 'ATAD3C', 'TMEM52', 'PRDM16', 'PER3', 'SLC45A1', 'SLC2A5', 'H6PD', 'KIF1B', 'KIAA2013', 'PRAMEF11', 'HNRNPCL1;HNRNPCL3;HNRNPCL4', 'HNRNPCL2', 'CELA3A', 'UBXN11', 'CRYBG2', 'KIAA1522', 'RNF19B', 'POU3F1', 'RLF', 'FOXO6', 'TMEM53', 'JAK1', 'CTBS', 'CELSR2', 'TRIM33', 'IGSF3', 'PDE4DIP', 'ANKRD34A', 'FAM231D;LOC100996741;LOC388692', 'TCHH', 'RPTN', 'HRNR', 'FLG', 'LCE4A', 'LCE1F', 'IVL', 'LOR', 'KCNN3', 'GBA', 'NES', 'LRRC71', 'OR10T2', 'SDHC', 'QSOX1', 'CFHR3', 'IGFN1', 'LMOD1', 'PIK3C2B', 'SLC26A9', 'SRGAP2', 'IARS2', 'LEFTY2', 'ITPKB', 'ZNF678', 'FMN2', 'WDR64', 'OR2T4', 'OR2T35', 'NOL10', 'EFR3B', 'ASXL2', 'ALK', 'STRN', 'PKDCC', 'SRBD1', 'C2orf73', 'BCL11A', 'FAM136A', 'ALMS1', 'REG3A', 'TCF7L1', 'TEKT4', 'ZNF2', 'ANKRD36C', 'ADRA2B', 'ANKRD36', 'ANKRD36B', 'POU3F3', 'ZC3H8', 'ZNF806', 'TMEM163', 'NEB', 'ITGA6', 'CIR1', 'FSIP2', 'FAM171B', 'AOX1', 'UNC80', 'CPS1', 'BARD1', 'SPEG', 'IRS1', 'MFF', 'SLC16A14', 'SPATA3', 'ARMC9', 'UGT1A7', 'ESPNL', 'AQP12B', 'AQP12A',